<a href="https://colab.research.google.com/github/muskaanramchandani/3o--Trabalho--Final-de-FIA/blob/main/TRABALHO_FINAL_DE_FIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch torchvision torchaudio --quiet
!pip install scikit-learn numpy matplotlib --quiet
!pip install git+https://github.com/logictensornetworks/LTNtorch.git --quiet



  Preparing metadata (setup.py) ... done


In [4]:
import torch
import ltn

print("Torch:", torch.__version__)
print("LTNtorch carregado com sucesso!")


Torch: 2.9.0+cpu
LTNtorch carregado com sucesso!


In [5]:
import numpy as np
import torch

def generate_objects(n=25):
    objs = []
    for _ in range(n):
        x, y = np.random.rand(), np.random.rand()
        color = np.eye(3)[np.random.randint(0, 3)]   # RGB
        shape = np.eye(5)[np.random.randint(0, 5)]   # 5 formas
        size = np.random.choice([0.0, 1.0])          # pequeno / grande
        objs.append(np.concatenate([[x, y], color, shape, [size]]))
    return torch.tensor(objs, dtype=torch.float32)

objects = generate_objects()
print(objects.shape)


torch.Size([25, 11])


/tmp/ipython-input-2329656342.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  return torch.tensor(objs, dtype=torch.float32)


In [6]:
import ltn

class Unary(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(11, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

class Binary(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(22, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x, y):
        return self.net(torch.cat([x, y], dim=1))


In [7]:
isCircle  = ltn.Predicate(Unary())
isSquare = ltn.Predicate(Unary())

leftOf   = ltn.Predicate(Binary())
rightOf  = ltn.Predicate(Binary())


In [8]:
x = ltn.Variable("x", objects)
y = ltn.Variable("y", objects)


In [11]:
# Operadores lógicos (LTNtorch oficial)

Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())

Forall = ltn.Quantifier(
    agg_op=ltn.fuzzy_ops.AggregPMeanError(p=2),
    quantifier="f"   # FOR ALL
)

Exists = ltn.Quantifier(
    agg_op=ltn.fuzzy_ops.AggregPMean(p=2),
    quantifier="e"   # EXISTS
)


In [13]:
axiom1 = Forall(x, Not(And(isCircle(x), isSquare(x))))
axiom2 = Forall(x, Or(isCircle(x), isSquare(x)))
axiom3 = Forall(x, Not(leftOf(x, x)))

axiom4 = Forall(
    [x, y],
    Implies(
        leftOf(x, y),
        Not(leftOf(y, x))
    )
)

axioms = [axiom1, axiom2, axiom3, axiom4]


In [14]:
satAgg = ltn.fuzzy_ops.AggregPMeanError(p=2)


In [17]:
optimizer = torch.optim.Adam(
    list(isCircle.model.parameters()) +
    list(isSquare.model.parameters()) +
    list(leftOf.model.parameters()),
    lr=0.001
)

for epoch in range(100):
    optimizer.zero_grad()

    # RECRIA os axiomas (grafo novo!)
    axiom1 = Forall(x, Not(And(isCircle(x), isSquare(x))))
    axiom2 = Forall(x, Or(isCircle(x), isSquare(x)))
    axiom3 = Forall(x, Not(leftOf(x, x)))
    axiom4 = Forall(
        [x, y],
        Implies(leftOf(x, y), Not(leftOf(y, x)))
    )

    axioms = [axiom1, axiom2, axiom3, axiom4]

    sat_values = torch.stack([axiom.value for axiom in axioms])
    loss = 1 - sat_values.mean()

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch} | SatAgg = {sat_values.mean().item():.4f}")

print("Treinamento finalizado")
print("Satisfatibilidade final:", sat_values.mean().item())



Epoch 0 | SatAgg = 0.7100
Epoch 20 | SatAgg = 0.7510
Epoch 40 | SatAgg = 0.7947
Epoch 60 | SatAgg = 0.8392
Epoch 80 | SatAgg = 0.8793
Treinamento finalizado
Satisfatibilidade final: 0.9130878448486328


In [18]:
def close_to_kernel(a, b, sigma=0.2):
    dist = torch.norm(a[:, :2] - b[:, :2], dim=1)
    return torch.exp(-(dist ** 2) / (2 * sigma ** 2))

class CloseTo(torch.nn.Module):
    def forward(self, x, y):
        return close_to_kernel(x, y).unsqueeze(1)

closeTo = ltn.Predicate(CloseTo())


In [19]:
inBetween = lambda x, y, z: Or(
    And(leftOf(y, x), rightOf(z, x)),
    And(leftOf(z, x), rightOf(y, x))
)


In [21]:
# Predicados de tamanho (baseados na última posição do vetor)

class IsSmall(torch.nn.Module):
    def forward(self, x):
        # size = 0 → pequeno
        return (1 - x[:, -1]).unsqueeze(1)

class IsBig(torch.nn.Module):
    def forward(self, x):
        # size = 1 → grande
        return x[:, -1].unsqueeze(1)

isSmall = ltn.Predicate(IsSmall())
isBig   = ltn.Predicate(IsBig())


In [22]:
class IsCylinder(torch.nn.Module):
    def forward(self, x):
        # shape index: [circle, square, cylinder, cone, triangle]
        return x[:, 5 + 2].unsqueeze(1)

isCylinder = ltn.Predicate(IsCylinder())


In [23]:
class Below(torch.nn.Module):
    def forward(self, x, y):
        # x abaixo de y se y está acima no eixo y
        return (x[:, 1] < y[:, 1]).float().unsqueeze(1)

class Above(torch.nn.Module):
    def forward(self, x, y):
        return (x[:, 1] > y[:, 1]).float().unsqueeze(1)

below = ltn.Predicate(Below())
above = ltn.Predicate(Above())


In [24]:
class RightOf(torch.nn.Module):
    def forward(self, x, y):
        return (x[:, 0] > y[:, 0]).float().unsqueeze(1)

rightOf = ltn.Predicate(RightOf())


In [26]:
z = ltn.Variable("z", objects)



In [28]:
query1 = Exists(
    x,
    And(
        isSmall(x),
        And(
            Exists(y, And(isCylinder(y), below(x, y))),
            Exists(z, And(isSquare(z), leftOf(x, z)))
        )
    )
)

print("Consulta 4.1 (sat):", query1.value.item())

print("Consulta 4.1 (sat):", query1.value.item())


Consulta 4.1 (sat): 0.0029654190875589848
Consulta 4.1 (sat): 0.0029654190875589848


In [29]:
# Predicado cor verde (índice 3 do vetor: [x,y, R, G, B, ...])
class IsGreen(torch.nn.Module):
    def forward(self, x):
        return x[:, 3].unsqueeze(1)

isGreen = ltn.Predicate(IsGreen())

query2 = Exists(
    [x, y, z],
    And(
        isCylinder(x),   # troque para isCone(x) se quiser exatamente cone
        And(
            isGreen(x),
            inBetween(x, y, z)
        )
    )
)

print("Consulta 4.2 (sat):", query2.value.item())


Consulta 4.2 (sat): 0.00014316638407763094


In [30]:
class IsTriangle(torch.nn.Module):
    def forward(self, x):
        # triangle é índice 4 dentro das shapes
        return x[:, 5 + 4].unsqueeze(1)

isTriangle = ltn.Predicate(IsTriangle())

class SameSize(torch.nn.Module):
    def forward(self, x, y):
        return (1 - torch.abs(x[:, -1] - y[:, -1])).unsqueeze(1)

sameSize = ltn.Predicate(SameSize())


In [31]:
axiom_proximity = Forall(
    [x, y],
    Implies(
        And(
            isTriangle(x),
            And(isTriangle(y), closeTo(x, y))
        ),
        sameSize(x, y)
    )
)

print("Regra proximidade (sat):", axiom_proximity.value.item())


Regra proximidade (sat): 0.9500164985656738


In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def gt_left(a, b):
    return (a[:, 0] < b[:, 0]).float()

y_true, y_pred = [], []

for i in range(len(objects)):
    for j in range(len(objects)):
        if i != j:
            a = objects[i].unsqueeze(0)
            b = objects[j].unsqueeze(0)

            # ground truth
            y_true.append(gt_left(a, b).item())

            # PREDIÇÃO PELO MODELO NEURAL (CORRETO)
            pred = leftOf.model(a, b).item()
            y_pred.append(pred > 0.5)

print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall   :", recall_score(y_true, y_pred))
print("F1 Score :", f1_score(y_true, y_pred))



Accuracy : 0.5
Precision: 0.0
Recall   : 0.0
F1 Score : 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [34]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def gt_left(a, b):
    return (a[:, 0] < b[:, 0]).float()

results = []

for run in range(5):
    print(f"\n=== RUN {run+1} ===")

    # 1. Geração de dados (passo 3.1.1)
    objects = generate_objects()

    # 2. Variáveis lógicas
    x = ltn.Variable("x", objects)
    y = ltn.Variable("y", objects)
    z = ltn.Variable("z", objects)

    # 3. Axiomas
    axiom1 = Forall(x, Not(And(isCircle(x), isSquare(x))))
    axiom2 = Forall(x, Or(isCircle(x), isSquare(x)))
    axiom3 = Forall(x, Not(leftOf(x, x)))
    axiom4 = Forall([x, y], Implies(leftOf(x, y), Not(leftOf(y, x))))

    axioms = [axiom1, axiom2, axiom3, axiom4]

    # 4. Satisfatibilidade (satAgg)
    sat = torch.stack([a.value for a in axioms]).mean().item()

    # 5. Métricas clássicas
    y_true, y_pred = [], []

    for i in range(len(objects)):
        for j in range(len(objects)):
            if i != j:
                a = objects[i].unsqueeze(0)
                b = objects[j].unsqueeze(0)

                y_true.append(gt_left(a, b).item())
                y_pred.append(leftOf.model(a, b).item() > 0.5)

    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec  = recall_score(y_true, y_pred)
    f1   = f1_score(y_true, y_pred)

    results.append({
        "Run": run + 1,
        "satAgg": sat,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    })



=== RUN 1 ===

=== RUN 2 ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



=== RUN 3 ===

=== RUN 4 ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



=== RUN 5 ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
df_results = pd.DataFrame(results)

print("\nResultados finais:")
df_results



Resultados finais:


,Run,satAgg,Accuracy,Precision,Recall,F1
0,1,0.917326,0.5,0.0,0.0,0.0
1,2,0.915547,0.5,0.0,0.0,0.0
2,3,0.920335,0.5,0.0,0.0,0.0
3,4,0.914760,0.5,0.0,0.0,0.0
4,5,0.917232,0.5,0.0,0.0,0.0


In [36]:
# salvar resultados
df_results.to_csv("resultados_ltn_5_execucoes.csv", index=False)
print("Arquivo salvo: resultados_ltn_5_execucoes.csv")


Arquivo salvo: resultados_ltn_5_execucoes.csv
